In [ ]:
%load_ext nb_black

In [1]:
# Import some python libraries
%matplotlib inline

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
from dask_gateway import Gateway
from dask.distributed import Client

gateway = Gateway()
options = gateway.cluster_options()
options.worker_cores = 4
cluster = gateway.new_cluster(cluster_options=options)

cluster.scale(48)
from time import sleep
sleep(10)

In [ ]:
client = Client(cluster)
client.restart()

---

# Introduction

This notebook demonstrates the performance of `climpred` on large datasets.
Here, we run `bootstrap_perfect_model` and `bootstrap_hindcast` `iteration` times
to calculate a pvalue that initialized skill surpasses uninitialized skill.

---

# perfect_model

## fake data

In [42]:
def gen_pm(degree=5, chunking_dim='lon'):
    ds = xr.Dataset()
    control = xr.Dataset()
    nmember = 10
    ninit = 12
    nlead = 5
    nx = 360//degree
    ny = 180//degree
    control_start = 3000
    control_end = 3300
    ntime = control_end - control_start

    times = xr.cftime_range(
        start=str(control_start),
        periods=ntime,
        freq='YS',
        calendar='noleap',
    )
    leads = np.arange(1, 1 + nlead)
    members = np.arange(1, 1 + nmember)
    inits = xr.cftime_range(
        start=str(control_start),
        periods=ninit,
        freq='10YS',
        calendar='noleap',
    )

    lons = xr.DataArray(
        np.linspace(0.5, 359.5, nx),
        dims=('lon',),
        attrs={'units': 'degrees east', 'long_name': 'longitude'},
    )
    lats = xr.DataArray(
        np.linspace(-89.5, 89.5, ny),
        dims=('lat',),
        attrs={'units': 'degrees north', 'long_name': 'latitude'},
    )
    ds['var'] = xr.DataArray(
                np.random.random(
                    (nmember, ninit, nlead, nx, ny),
                ),
                coords={
                    'member': members,
                    'init': inits,
                    'lon': lons,
                    'lat': lats,
                    'lead': leads,
                },
                dims=('member', 'init', 'lead', 'lon', 'lat'),
                name='var',)

    control['var'] = xr.DataArray(
                np.random.random((ntime, nx, ny)),
                coords={'lon': lons, 'lat': lats, 'time': times},
                dims=('time', 'lon', 'lat'),
                name='var',
                attrs={'units': 'var units', 'description': 'a description'},
            )
    control=control['var'].chunk({'time':'auto'})

    ds.lead.attrs['units'] = 'years'

    ds=ds['var']
    if chunking_dim is not None:
        ds=ds.chunk({chunking_dim:'auto'})
        control=control.chunk({chunking_dim:'auto'})

    return ds, control

ds, control = gen_pm()

## Skill

In [43]:
from climpred.prediction import compute_perfect_model

In [44]:
cp_kwargs = {'metric': 'acc', 'comparison':'m2e'}

In [48]:
ds.data

dask.array<xarray-<this-array>, shape=(10, 12, 5, 72, 36), dtype=float64, chunksize=(10, 12, 5, 72, 36), chunktype=numpy.ndarray>

In [47]:
%time skill = compute_perfect_model(ds, control, **cp_kwargs)
skill.data

CPU times: user 120 ms, sys: 5.04 ms, total: 125 ms
Wall time: 122 ms


dask.array<pearson_r, shape=(5, 72, 36), dtype=float64, chunksize=(5, 72, 36), chunktype=numpy.ndarray>

In [49]:
%time skillc = skill.compute()

CPU times: user 28 ms, sys: 11.3 ms, total: 39.3 ms
Wall time: 275 ms


## Bootstrap significant skill

In [50]:
from climpred.bootstrap import bootstrap_perfect_model

In [54]:
iterations=500

In [55]:
%time bskill = bootstrap_perfect_model(ds, control, iterations=iterations, **cp_kwargs)
bskill.data

/srv/conda/envs/notebook/lib/python3.7/site-packages/dask/array/core.py:342: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)


CPU times: user 5.02 s, sys: 91.2 ms, total: 5.11 s
Wall time: 5.44 s


dask.array<concatenate, shape=(4, 3, 5, 72, 36), dtype=float64, chunksize=(2, 1, 1, 72, 36), chunktype=numpy.ndarray>

In [56]:
improved_by_init = bskill.sel(results='skill',kind='init').where(bskill.sel(results='p',kind='uninit') <= 0.05)
%time improved_by_init_c = improved_by_init.compute()

CPU times: user 6.71 s, sys: 736 ms, total: 7.44 s
Wall time: 4min 42s


# Hindcast

## fake data

In [82]:
def gen_hind(degree=5,nmember=34,nlead=10, chunking_dim='lon'):
    hind = xr.Dataset()
    observations = xr.Dataset()
    uninit=xr.Dataset()
    nx = 360//degree
    ny = 180//degree
    init_start = 1960
    init_end = 2015
    ninit = init_end - init_start

    times = xr.cftime_range(
        start=str(init_start),
        periods=ninit,
        freq='YS',
        calendar='noleap',
    )
    leads = np.arange(1, 1 + nlead)
    members = np.arange(1, 1 + nmember)
    inits = xr.cftime_range(
        start=str(init_start), end=str(init_end-1),
        freq='YS',
    )

    lons = xr.DataArray(
        np.linspace(0.5, 359.5, nx),
        dims=('lon',),
        attrs={'units': 'degrees east', 'long_name': 'longitude'},
    )
    lats = xr.DataArray(
        np.linspace(-89.5, 89.5, ny),
        dims=('lat',),
        attrs={'units': 'degrees north', 'long_name': 'latitude'},
    )
    hind['var'] = xr.DataArray(
                np.random.random(
                    (nmember, ninit, nlead, nx, ny),
                ),
                coords={
                    'member': members,
                    'init': inits,
                    'lon': lons,
                    'lat': lats,
                    'lead': leads,
                },
                dims=('member', 'init', 'lead', 'lon', 'lat'),
                name='var',)

    observations['var'] = xr.DataArray(
                np.random.random((ninit, nx, ny)),
                coords={'lon': lons, 'lat': lats, 'time': inits},
                dims=('time', 'lon', 'lat'),
                name='var',
                attrs={'units': 'var units', 'description': 'a description'},
            )
    uninit['var'] = xr.DataArray(
                np.random.random((ninit, nx, ny, nmember)),
                coords={'lon': lons, 'lat': lats, 'time': inits, 'member': members},
                dims=('time', 'lon', 'lat', 'member'),
                name='var')

    if chunking_dim is not None:
        observations=observations.chunk({chunking_dim:'auto'})
        uninit=uninit.chunk({chunking_dim:'auto'})
        hind=hind.chunk({chunking_dim:'auto'})
        hind=hind.chunk({'lon':-1})
        hind=hind.chunk({'lead':1})

    observations=observations['var']
    uninit=uninit['var']
    hind=hind['var']
    hind.lead.attrs['units'] = 'years'

    return hind,uninit,observations

hind,hist,obs = gen_hind()

## Skill

In [83]:
from climpred.prediction import compute_hindcast

In [84]:
cp_kwargs = {'metric': 'acc', 'comparison':'e2r'}

In [85]:
hind.data

dask.array<rechunk-merge, shape=(34, 55, 10, 72, 36), dtype=float64, chunksize=(34, 55, 1, 72, 36), chunktype=numpy.ndarray>

In [90]:
%time skill = compute_hindcast(hind, obs, **cp_kwargs)
skill.data

CPU times: user 347 ms, sys: 4.46 ms, total: 351 ms
Wall time: 348 ms


dask.array<concatenate, shape=(10, 72, 36), dtype=float64, chunksize=(1, 72, 36), chunktype=numpy.ndarray>

In [91]:
%time skillc = skill.compute()

CPU times: user 653 ms, sys: 226 ms, total: 879 ms
Wall time: 375 ms


## Bootstrap significant skill

In [35]:
from climpred.bootstrap import bootstrap_hindcast

In [36]:
iterations=100

In [38]:
%time bskill = bootstrap_hindcast(hind, hist, obs, iterations=iterations, **cp_kwargs)
bskill.data

/srv/conda/envs/notebook/lib/python3.7/site-packages/dask/array/core.py:342: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)


CPU times: user 3.98 s, sys: 660 ms, total: 4.64 s
Wall time: 5.97 s


dask.array<concatenate, shape=(4, 3, 10, 72, 36), dtype=float64, chunksize=(2, 1, 1, 72, 36), chunktype=numpy.ndarray>

In [ ]:
improved_by_init = bskill.sel(results='skill',kind='init').where(bskill.sel(results='p',kind='uninit') <= 0.05)
%time improved_by_init_c = improved_by_init.compute()

## classes

In [58]:
from climpred.classes import HindcastEnsemble
h = HindcastEnsemble(hind)
h = h.add_observations(obs, 'obs')
h = h.add_uninitialized(hist)
h

<climpred.HindcastEnsemble>
Initialized Ensemble:
    var      (member, init, lead, lon, lat) float64 dask.array<chunksize=(34, 55, 10, 72, 36), meta=np.ndarray>
obs:
    var      (time, lon, lat) float64 dask.array<chunksize=(55, 72, 36), meta=np.ndarray>
Uninitialized:
    var      (time, lon, lat, member) float64 dask.array<chunksize=(55, 72, 36, 34), meta=np.ndarray>

In [64]:
iterations=100
metric='acc'
comparison='e2r'

In [61]:
from climpred.bootstrap import _resample_iterations
resample = _resample_iterations

In [62]:
h._datasets['initialized'] = resample(hind.to_dataset(), iterations).chunk({'lead':1,'lat':'auto','lon':'auto'})
h._datasets['initialized']['var'].data

,Array,Chunk
Bytes,38.78 GB,38.78 MB
Shape,"(100, 34, 55, 10, 72, 36)","(1, 34, 55, 1, 72, 36)"
Count,2305 Tasks,1000 Chunks
Type,float64,numpy.ndarray


In [65]:
bootstrapped_init_skill = h.verify(metric=metric,comparison=comparison)
bootstrapped_init_skill['var'].data

,Array,Chunk
Bytes,20.74 MB,20.74 kB
Shape,"(10, 100, 72, 36)","(1, 1, 72, 36)"
Count,11512 Tasks,1000 Chunks
Type,float64,numpy.ndarray


In [67]:
h._datasets['uninitialized'] = resample(hist, iterations).to_dataset().chunk({'time':'auto','lat':'auto','lon':'auto','iteration':'auto'})
h._datasets['uninitialized']['var'].data

,Array,Chunk
Bytes,3.88 GB,77.55 MB
Shape,"(100, 55, 72, 36, 34)","(2, 55, 72, 36, 34)"
Count,351 Tasks,50 Chunks
Type,float64,numpy.ndarray


In [68]:
bootstrapped_skill = h.verify(metric=metric,reference='historical')
bootstrapped_skill['var'].data

,Array,Chunk
Bytes,41.47 MB,20.74 kB
Shape,"(2, 10, 100, 72, 36)","(1, 1, 1, 72, 36)"
Count,18263 Tasks,2000 Chunks
Type,float64,numpy.ndarray


In [69]:
from climpred.bootstrap import _pvalue_from_distributions
from climpred.comparisons import ALL_COMPARISONS, COMPARISON_ALIASES, HINDCAST_COMPARISONS
from climpred.metrics import ALL_METRICS, METRIC_ALIASES
from climpred.utils import get_comparison_class,get_metric_class
metric = METRIC_ALIASES.get(metric, metric)
comparison = COMPARISON_ALIASES.get(comparison, comparison)

metric = get_metric_class(metric, ALL_METRICS)
comparison = get_comparison_class(comparison, ALL_COMPARISONS)

In [75]:
p = _pvalue_from_distributions(
        bootstrapped_skill.sel(skill='historical'), bootstrapped_skill.sel(skill='init'), metric=metric
    )
p['var'].data

dask.array<truediv, shape=(10, 72, 36), dtype=float64, chunksize=(1, 72, 36), chunktype=numpy.ndarray>

In [77]:
%time _ = p.compute()

CPU times: user 2.36 s, sys: 275 ms, total: 2.64 s
Wall time: 1min 56s


# Close down

In [ ]:
client.close()
cluster.close()